<a href="https://colab.research.google.com/github/hamzaimran01/Deep-Learning-Lab/blob/master/Hiwi_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
print(torch.__version__)
import numpy as np
from torch.autograd import Variable
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
!curl https://raw.githubusercontent.com/automl/smac3/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install smac

1.0.1.post2
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   220  100   220    0     0   1037      0 --:--:-- --:--:-- --:--:--  1037
  Using cached https://files.pythonhosted.org/packages/ed/0f/4d7e42a9dfef3a1898e03cffa8f1cfcd1f96507d718808b2db584c6f8401/pyrfr-0.8.0.tar.gz
  Stored in directory: /root/.cache/pip/wheels/61/1a/d2/b5aee388a492a01946143d3c976b2ca810af537480e1f16999
Successfully built pyrfr


In [0]:

import typing



from smac.facade.borf_facade import BORF
from smac.scenario.scenario import Scenario
from smac.configspace import ConfigurationSpace
from smac.runhistory.runhistory import RunKey
from smac.tae.execute_func import ExecuteTAFuncArray

from ConfigSpace.hyperparameters import UniformFloatHyperparameter
from smac.facade.func_facade import fmin_smac
def branin(x):
    x1 = x[0]
    x2 = x[1]
    a = 1.
    b = 5.1 / (4.*np.pi**2)
    c = 5. / np.pi
    r = 6.
    s = 10.
    t = 1. / (8.*np.pi)
    ret = a*(x2-b*x1**2+c*x1-r)**2+s*(1-t)*np.cos(x1)+s
    return ret

In [32]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import numpy as np 

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
import os

import requests
import numpy as np
from torch.utils.data import Dataset


class KMNIST(Dataset):
    """
    Dataset class for use with pytorch for the Kuzushiji-MNIST dataset as given in
    Deep Learning for Classical Japanese Literature. Tarin Clanuwat et al. arXiv:1812.01718

    Kuzushiji-MNIST contains 70,000 28x28 grayscale images spanning 10 classes (one from each column of hiragana),
    and is perfectly balanced like the original MNIST dataset (6k/1k train/test for each class).
    """

    def __init__(self, data_dir='.', train: bool = True, transform=None):
        """
        :param data_dir: Directory of the data
        :param train: Use training or test set
        :param transform: pytorch transforms for data augmentation
        """

        self.__urls = [
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz',
        ]

        t_str = 'train' if train else 'test'
        imgs_fn = 'kmnist-{}-imgs.npz'.format(t_str)
        labels_fn = 'kmnist-{}-labels.npz'.format(t_str)

        if not os.path.exists(data_dir):
            os.mkdir(os.path.abspath(data_dir))
        if not os.path.exists(os.path.abspath(os.path.join(data_dir, imgs_fn))):
            self.__download(os.path.abspath(data_dir))

        imgs_fn = os.path.abspath(os.path.join(data_dir, imgs_fn))
        labels_fn = os.path.abspath(os.path.join(data_dir, labels_fn))
       
        self.images = np.load(imgs_fn)['arr_0']
        self.labels = np.load(labels_fn)['arr_0']
        self.n_classes = len(np.unique(self.labels))
       
        
        self.class_labels, self.class_frequency = np.unique(self.labels, return_counts=True)
        self.class_frequency = self.class_frequency / np.sum(self.class_frequency)
        self.data_dir = data_dir
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1  # only gray scale
        self.transform = transform

    def __len__(self):
       
        return len(self.labels)

    def __getitem__(self, idx):#image size is 28*28*1
        image = np.expand_dims(self.images[idx], axis=-1)
        if self.transform:
            image = self.transform(image)
            
            

        label = np.int(self.labels[idx])
       # var=torch.tensor(image)
        #a = np.squeeze(var)
        #plt.imshow(a, cmap='Greys')
        #print(a.size())
       
        return image, label

    def __download(self, data_dir):
        print('Datadir', data_dir)
   
        for url in self.__urls:
            fn = os.path.basename(url)
            req = requests.get(url, stream=True)
            print('Downloading {}'.format(fn))
            with open(os.path.join(data_dir, fn), 'wb') as fh:
                for chunck in req.iter_content(chunk_size=1024):
                    if chunck:
                        fh.write(chunck)
            print('done')
        print('All files downloaded')
        
    

    
    

    


        

      

In [0]:
#tensor shape and distributed into train and test sets


transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

train_set=torchvision.datasets.KMNIST(root='./KMNIST', train=True, download=True, transform=transform)


test_set = torchvision.datasets.KMNIST(root='./KMNIST', train=False, download=True, transform=transform)


  



In [0]:
#basically uses the data in the iteration
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,sampler=train_sampler)
    return(train_loader)
  
def get_test_loader(batch_size):
    test_loader=torch.utils.data.DataLoader(test_set, batch_size=batch_size,shuffle=False)
    return(test_loader)
  

In [0]:
#forming samplers of train,test 

from torch.utils.data.sampler import SubsetRandomSampler

#Training
n_training_samples = 30000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))



#Test
n_test_samples = 1000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

In [0]:
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 1, output channels = 18
        self.conv1 = torch.nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 14 * 14, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
      
        
        
        #Computes the activation of the first convolution
        #Size changes from (3, 28, 28) to (18, 28, 28)
        x = F.relu(self.conv1(x))
     
        
        #Size changes from (18, 28, 28) to (18, 14, 14)
        x = self.pool(x)
        
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 14, 14) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 14 *14)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [0]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)
  
  
  
def SMAC(net,func):  
  x, cost, _ = fmin_smac(func=branin,x0=[0, 0],bounds=[(-5, 10), (0, 15)],maxfun=500,rng=3)        
  print("Optimum at {} with cost of {}".format(x, cost)),
  
  return x,cost,_

In [0]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    #print("the number of batches are ",n_batches)
    
    
    #Create our loss and optimizer functions
    optimizer,loss = SMAC(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
           
           
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.item()
            total_train_loss += loss_size.item()
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
       # total_val_loss = 0
        #for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
         #   inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
          #  val_outputs = net(inputs)
           # val_loss_size = loss(val_outputs, labels)
            #total_val_loss += val_loss_size.data[0]
            
        #print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
#print("Training finished, took {:.2f}s".format(time.time() - training_start_time()))

In [0]:

CNN = SimpleCNN()
trainNet(CNN, batch_size=28, n_epochs=2, learning_rate=0.001)

In [42]:

correct = 0
total = 0

testloader=get_test_loader(28)
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
    
        images_test, labels_test = data
        inputs, labels = Variable(images_test), Variable(labels_test)
            
        outputs=CNN(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels_test.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 9 %
